In [ ]:
#generate bm25+ltr&d2q(20)
from collections import defaultdict
counts = defaultdict(int)
with open('run.ltr.d2q_field.20.tsv', 'w') as output_f:
    with open('run.ltr.d2q_field.1000.tsv') as input_f:
        for line in input_f:
            cols = line.split('\t')
            qid = cols[0]
            if(counts[qid] < 20):
                docid = cols[1]
                counts[qid] += 1
                output_f.write(f'{qid}\t{docid}\t{counts[qid]}\n')

In [ ]:
!python3 ../RA/pyserini/tools/scripts/msmarco/msmarco_passage_eval.py \
 ../RA/pyserini/collections/msmarco-passage/qrels.dev.small.tsv ./run.ltr.d2q_field.20.tsv

#####################
MRR @10: 0.31363498885705093
QueriesRanked: 6980
#####################


In [ ]:
!python3 ../RA/pyserini/tools/scripts/msmarco/convert_msmarco_to_trec_run.py --input ./run.ltr.d2q_field.20.tsv --output ./run.ltr.d2q_field.20.trec

Done!


In [ ]:
!../RA/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mmap -m ndcg_cut.10\
 ../RA/pyserini/collections/msmarco-passage/qrels.dev.small.trec ./run.ltr.d2q_field.20.trec

map                   	all	0.3152
ndcg_cut_10           	all	0.3776


In [ ]:
#get what qid,pid pairs are in ltr top 20
from tqdm import tqdm
ltr = defaultdict(dict)
with open('./run.ltr.d2q_field.20.tsv') as f_ltr:
    for line in tqdm(f_ltr):
        query_id, passage_id, rank = line.strip().split('\t')
        if query_id not in ltr:
            ltr[query_id] = [passage_id]
        else:
            ltr[query_id].append(passage_id)

139596it [00:00, 874098.76it/s]


In [ ]:
bert = defaultdict(dict)
with open('../Downloads/run.monot5.d2q_field_1000.tsv') as f_bert, open('./run.monot5.ltr.d2q_field.20.tsv','w') as fout:    
    for line in tqdm(f_bert):
        query_id, passage_id, t = line.strip().split('\t')
        bert[query_id][passage_id] = int(t)
    for qid, pids in tqdm(ltr.items()):
        score = [(pid, bert[qid][pid]) for pid in pids]
        score = sorted(score, key=lambda x:x[1])
        rank = 1
        for pid, _ in score:
            fout.write(f'{qid}\t{pid}\t{rank}\n')
            rank += 1

6974598it [00:09, 753043.37it/s]
100%|██████████| 6980/6980 [00:00<00:00, 34452.97it/s]


In [ ]:
!python3 ../RA/pyserini/tools/scripts/msmarco/msmarco_passage_eval.py \
 ../RA/pyserini/collections/msmarco-passage/qrels.dev.small.tsv ./run.monot5.ltr.d2q_field.20.tsv

#####################
MRR @10: 0.3823739596124986
QueriesRanked: 6980
#####################


In [ ]:
!python3 ../RA/pyserini/tools/scripts/msmarco/convert_msmarco_to_trec_run.py --input ./run.monot5.ltr.d2q_field.20.tsv --output ./run.monot5.ltr.d2q_field.20.trec

Done!


In [ ]:
!../RA/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mmap -m ndcg_cut.10\
 ../RA/pyserini/collections/msmarco-passage/qrels.dev.small.trec ./run.monot5.ltr.d2q_field.20.trec

map                   	all	0.3799
ndcg_cut_10           	all	0.4435
